One vs All Method

Train NMF for each topic separately.

Use all Wiki articles as Background Corpus.

In [286]:
import pandas as pd
import numpy as np
from time import time

import nltk
from nltk.corpus import brown
from nltk.tokenize.moses import MosesDetokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

import matplotlib.pyplot as plt
from math import pi

from omterms.interface import *

from ipywidgets import interact, fixed

import pickle

## Plots and Prints

In [287]:
categories=['universalism', 'hedonism', 'achievement', 'power',
       'self-direction', 'benevolence', 'conformity', 'tradition', 'stimulation',
       'security']

schwartz =['universalism', 'benevolence', 'conformity', 'tradition',
       'security', 'power', 'achievement', 'hedonism', 'stimulation',
       'self-direction']

## Helper Functions

In [288]:
def cumulate_W(W, n_topics):
    W_cumul = []
    for d in W:
        temp = []
        for i in range(W.shape[1]//n_topics):
            temp.append(d[i*n_topics:(i+1)*n_topics].sum())
        W_cumul.append(temp)

    W_cumul = np.asarray(W_cumul)
    
    return W_cumul

def normalize_W(W):
    W_cumul_norm = W/(W.sum(axis=1).reshape(W.shape[0], 1))
    W_cumul_norm *= 100
    
    return W_cumul_norm

def export_to_excel(W, data, filepath):
    '''
    Take cumulated W as input.
    Don't forget to put xlsx as file extension '''
    
    df = pd.DataFrame(data=W,index = range(len(W)), columns=schwartz)
    df['document.id'] = data['document.id']
    df['title'] = data['title']
    df['theme'] = data['theme']
    
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    df = df[cols]
    
    df.to_excel(filepath)
    return df

def export_to_csv(W, data, filepath):
    '''
    Take cumulated W as input.
    Don't forget to put csv as file extension '''
    
    df = pd.DataFrame(data=W,index = range(len(W)), columns=categories)
    df['document.id'] = data['document.id']
    df['title'] = data['title']
    df['theme'] = data['theme']
    
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    df = df[cols]

    df.to_csv(filepath)
    return df

## Main Functions

In [289]:
def read_data(filepath):
    data = pd.read_json(filepath)
    data = data[data['text']!=""]
    data = data.sort_values('theme.id')
    data.reset_index(drop=True,inplace=True)
    
    return data
    
def extract_corpus(data):    
    corpus = list(data['text'])
    return corpus

def preprocess_corpus(corpus):
    PPcorpus = [' '.join(list((extract_terms(doc, extra_process = ['stem'])['Stem']+' ')*extract_terms(doc, extra_process = ['stem'])['TF'])) for doc in corpus]
    return PPcorpus

def train_corpus(corpus, data, brown_corpus, n_topics=3, betaloss = 'kullback-leibler', bckg_brown = False):
    N = len(data)
    
    theme_counts = data.groupby(['theme.id','theme']).count().iloc[:,1]
    pd_theme_counts = pd.DataFrame(theme_counts)
    n_themes = len(theme_counts)
    
    n_top_words = 5
    n_components = n_topics*(n_themes)
    
    
    tfidf_vectorizer = TfidfVectorizer() # optionally add maxfeatures = n_features to enforce number of features
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(corpus+brown_corpus)
    n_features = tfidf.shape[1]
    
    W_list = []
    
    if bckg_brown:
        tc_sum = 0
        for tc in theme_counts:
            W = np.zeros((N+len(brown_corpus),2*n_topics))
            W[N:, n_topics:] = np.random.random((len(brown_corpus),n_topics))
            W[tc_sum:tc_sum+tc, :] = np.random.random((tc,2*n_topics))

            tc_sum += tc
            W_list.append(W)
    else:
        tc_sum = 0
        for tc in theme_counts:
            W = np.zeros((N,2*n_topics))
            W[:, n_topics:] = np.random.random((N,n_topics))
            W[tc_sum:tc_sum+tc, :n_topics] = np.random.random((tc,n_topics))

            tc_sum += tc
            W_list.append(W)
        
    X = tfidf 
    nmf_list = []

    for i, W in enumerate(W_list):
        t0 = time()
        H = np.random.rand(2*n_topics, n_features)

        nmf = NMF(n_components= 2*n_topics, solver='mu', beta_loss=betaloss,
                  alpha=.1, l1_ratio=.5, init = 'custom')

        nmf.fit_transform(X=X,W=W,H=H)

        nmf_list.append(nmf)
    
    
    return nmf_list, W_list, tfidf, tfidf_vectorizer
    
def evaluate_test_corpus(test_corpusPP, nmf_list, tfidf_vectorizer):
    t0 = time()
    tfidf_test = tfidf_vectorizer.transform(test_corpusPP)
    #tfidf = tfidf_vectorizer.transform(corpusX)
    #print(tfidf_test.shape[1])
    n_features = tfidf_test.shape[1]

    W_test_list = []
    for nmf in nmf_list:
        W_test = evaluate_docs(test_corpusPP, nmf, tfidf_test, betaloss = 'kullback-leibler')
        W_test_list.append(W_test)
        
    # Sum up sub topics
    W_test_norm_list = []
    for W in W_test_list:
        W_test_cumul = cumulate_W(W, n_topics=3)
        W_test_norm = normalize_W(W_test_cumul)
        W_test_norm_list.append(W_test_norm)
    W_test_norm = np.asarray(W_test_norm_list).T[0]
    W_test_norm = np.nan_to_num(W_test_norm)

    # cumulated-normalized and raw
    return W_test_norm

def evaluate_docs(docs, nmf, tfidf_test, betaloss = 'kullback-leibler'):
    X_test = tfidf_test
    H_test = nmf.components_
    
    # Fit the NMF model
    t0 = time()

    W_test = nmf.transform(X_test)
    
    return W_test

## Leave-One-Out Testing

In [11]:
#https://github.com/bulentozel/OpenMaker/blob/master/Semantics/data/corpuses/schwartz.json
# schwartz.json or pruned_schwartz.json
filepath = 'pruned_schwartz.json'

data = read_data(filepath)
corpus = extract_corpus(data)
corpusPP = preprocess_corpus(corpus)

Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 3909
Cleaning process: Initial size of tokens = 3909
Reduction due to punctuations and stopwords = 2792.
Reduction due to all numeral terms = 8
Reduction due to short terms = 8
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 3
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 2811
Percentage = 72%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom sto

Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 724
Cleaning process: Initial size of tokens = 724
Reduction due to punctuations and stopwords = 496.
Reduction due to all numeral terms = 1
Reduction due to short terms = 2
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 0
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 499
Percentage = 69%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...


Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 13100
Cleaning process: Initial size of tokens = 13100
Reduction due to punctuations and stopwords = 9653.
Reduction due to all numeral terms = 25
Reduction due to short terms = 11
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 18
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 9707
Percentage = 74%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cle

Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 10721
Cleaning process: Initial size of tokens = 10721
Reduction due to punctuations and stopwords = 8545.
Reduction due to all numeral terms = 16
Reduction due to short terms = 3
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 7
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 8571
Percentage = 80%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom 

Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 1914
Cleaning process: Initial size of tokens = 1914
Reduction due to punctuations and stopwords = 1343.
Reduction due to all numeral terms = 5
Reduction due to short terms = 3
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 4
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1355
Percentage = 71%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom sto

...

Done. Number of terms: 4063
Cleaning process: Initial size of tokens = 4063
Reduction due to punctuations and stopwords = 2951.
Reduction due to all numeral terms = 7
Reduction due to short terms = 6
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 1
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 2965
Percentage = 73%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 4063
Cleaning process: Initial size of tokens = 4063
Reduction due to punctuations and stopwords = 2

Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2853
Cleaning process: Initial size of tokens = 2853
Reduction due to punctuations and stopwords = 2013.
Reduction due to all numeral terms = 15
Reduction due to short terms = 13
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 2
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 2043
Percentage = 72%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleane

Percentage = 74%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 6117
Cleaning process: Initial size of tokens = 6117
Reduction due to punctuations and stopwords = 4521.
Reduction due to all numeral terms = 8
Reduction due to short terms = 4
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 11
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 4544
Percentage = 74%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insi

Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 5957
Cleaning process: Initial size of tokens = 5957
Reduction due to punctuations and stopwords = 4266.
Reduction due to all numeral terms = 6
Reduction due to short terms = 6
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 9
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 4287
Percentage = 72%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner 

Done. Number of terms: 7966
Cleaning process: Initial size of tokens = 7966
Reduction due to punctuations and stopwords = 6140.
Reduction due to all numeral terms = 2
Reduction due to short terms = 5
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 4
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 6151
Percentage = 77%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2219
Cleaning process: Initial size of tokens = 2219
Reduction due to punctuations and stopwords = 1

In [12]:
mdetok = MosesDetokenizer()

brown_files_sent = []
for fid in brown.fileids():
    brown_files_sent.append([mdetok.detokenize(' '.join(sent).replace('``', '"').replace("''", '"').replace('`', "'").split(), return_str=True)  for sent in brown.sents(fid)])
    
brown_natural = [' '.join(bfs) for bfs in brown_files_sent]
brown_naturalPP = preprocess_corpus(brown_natural)

Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2021
Cleaning process: Initial size of tokens = 2021
Reduction due to punctuations and stopwords = 1326.
Reduction due to all numeral terms = 26
Reduction due to short terms = 6
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 1
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1359
Percentage = 67%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom st

Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2038
Cleaning process: Initial size of tokens = 2038
Reduction due to punctuations and stopwords = 1358.
Reduction due to all numeral terms = 19
Reduction due to short terms = 5
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 2
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1384
Percentage = 68%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner

Reduction due to short terms = 10
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 1
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1355
Percentage = 66%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2129
Cleaning process: Initial size of tokens = 2129
Reduction due to punctuations and stopwords = 1370.
Reduction due to all numeral terms = 35
Reduction due to short terms = 8
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 1
Reduction d

Reduction due to rare terms = 0
Reduction due to partially numeral terms = 0
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1441
Percentage = 69%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2100
Cleaning process: Initial size of tokens = 2100
Reduction due to punctuations and stopwords = 1238.
Reduction due to all numeral terms = 16
Reduction due to short terms = 14
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 0
Reduction due to terms with not allowed symb

Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2066
Cleaning process: Initial size of tokens = 2066
Reduction due to punctuations and stopwords = 1240.
Reduction due to all numeral terms = 44
Reduction due to short terms = 9
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 2
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1295
Percentage = 63%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner

...

Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2032
Cleaning process: Initial size of tokens = 2032
Reduction due to punctuations and stopwords = 1386.
Reduction due to all numeral terms = 0
Reduction due to short terms = 0
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 0
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1386
Percentage = 68%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom sto

Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2051
Cleaning process: Initial size of tokens = 2051
Reduction due to punctuations and stopwords = 1447.
Reduction due to all numeral terms = 1
Reduction due to short terms = 1
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 1
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1450
Percentage = 71%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner 

Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2058
Cleaning process: Initial size of tokens = 2058
Reduction due to punctuations and stopwords = 1463.
Reduction due to all numeral terms = 0
Reduction due to short terms = 0
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 0
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1463
Percentage = 71%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom sto

Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2062
Cleaning process: Initial size of tokens = 2062
Reduction due to punctuations and stopwords = 1361.
Reduction due to all numeral terms = 2
Reduction due to short terms = 0
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 0
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1363
Percentage = 66%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner 

Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 2073
Cleaning process: Initial size of tokens = 2073
Reduction due to punctuations and stopwords = 1521.
Reduction due to all numeral terms = 0
Reduction due to short terms = 0
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 0
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1521
Percentage = 73%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom sto

In [18]:
t0 = time()

all_W_norm = []
for i in range(0, 90):
    print(str(i) + " - " + data.iloc[i]['theme'] + " - " + str(time()-t0))
    nmf_list, W_list, tfidf, tfidf_vectorizer = train_corpus(corpusPP[:i]+corpusPP[i+1:], data.drop(i), brown_naturalPP, n_topics=3, betaloss = 'kullback-leibler', bckg_brown = True)
    W_test_norm = evaluate_test_corpus([corpusPP[i]], nmf_list, tfidf_vectorizer)
    all_W_norm.append(W_test_norm.squeeze())



0 - universalism - 0.0009963512420654297
1 - universalism - 151.00201153755188
2 - universalism - 298.9113140106201
3 - universalism - 448.8950352668762
4 - universalism - 597.532372713089
5 - universalism - 751.4256489276886
6 - universalism - 900.3532128334045
7 - universalism - 1052.4433121681213
8 - universalism - 1194.88223528862
9 - universalism - 1347.0122389793396
10 - universalism - 1494.7559638023376
11 - universalism - 1641.9242367744446
12 - universalism - 1797.1539342403412
13 - universalism - 1942.9877722263336
14 - universalism - 2103.510313272476
15 - universalism - 2251.308896780014
16 - universalism - 2398.035855293274
17 - universalism - 2548.119326353073
18 - universalism - 2688.795960664749
19 - universalism - 2836.2494666576385
20 - universalism - 2963.7094626426697
21 - universalism - 3128.5514466762543
22 - universalism - 3274.240197658539
23 - universalism - 3420.529816865921
24 - universalism - 3563.0076339244843
25 - universalism - 3720.26690530777
26 - unive

In [296]:
all_W_norm = pickle.load(open( "loo_all_W_norm_brown.p", "rb" ) )
#all_W_norm = pickle.load(open( "loo_all_W_norm.p", "rb" ) )

In [297]:
all_W_norm = np.asarray(all_W_norm)
schwartz_dist = []
for doc in range(len(all_W_norm)):
    temp_dist = []
    for sch in schwartz:
        temp_dist.append(all_W_norm[doc][categories.index(sch)])
    schwartz_dist.append(temp_dist)
schwartz_dist = np.asarray(schwartz_dist)

In [298]:
df = export_to_excel(schwartz_dist, data, filepath = 'leave-one-out-test-brown.xlsx')
df.head()

,document.id,title,theme,universalism,benevolence,conformity,tradition,security,power,achievement,hedonism,stimulation,self-direction
0,1,Critical thinking,universalism,89.653034,91.377110,91.767514,86.400926,91.539732,87.743044,93.157306,84.538290,89.929512,91.926951
1,221,Social work,universalism,92.050226,80.640644,79.729339,71.130561,93.415741,82.973158,92.096706,68.992499,80.146128,90.190076
2,222,Labor rights,universalism,93.429417,85.060399,83.418897,81.072764,92.420685,89.237878,92.642613,70.249667,94.400966,88.771561
3,223,Left-wing politics,universalism,94.871711,84.798562,75.561284,82.453156,87.946132,93.183788,87.861797,76.827805,81.134122,85.091639
4,224,Climate justice,universalism,94.723394,67.553533,79.514097,56.967175,95.548738,75.354752,86.980349,46.831929,83.728828,82.912628


In [114]:
df = export_to_csv(all_W_norm, data, filepath = 'output.csv')
df.head()

,document.id,title,theme,universalism,benevolence,conformity,tradition,security,power,achievement,hedonism,stimulation,self-direction
0,1,Critical thinking,universalism,60.910666,49.848646,52.946681,22.593145,25.246669,28.323066,63.209017,9.742318,0.120587,41.536684


### Analysis
* Skoru en yuksek olan ve bu skora 5 yaklasik olan temalar
* Skoru en dusuk olan ve bu skora 5 yaklasik olan temalar
* Kendi temasi 80'in uzerinde olanlar
* En yuksekler ve En dusukler icinde kendi temasi veya zit temasi bulunanlar.

In [266]:
schwartz =['universalism', 'benevolence', 'conformity', 'tradition',
       'security', 'power', 'achievement', 'hedonism', 'stimulation',
       'self-direction']

schwartz_hier = {
    'self-transcendence': ['universalism', 'benevolence'],
    'conservation': ['conformity', 'tradition','security'],
    'self-enhancement': ['power', 'achievement'],
    'hhedonism': ['hedonism'],
    'opennes-to-change': ['stimulation','self-direction']
}

schwartz_hier_pos = {
    'universalism': 'self-transcendence',
    'benevolence': 'self-transcendence',
    'conformity': 'conservation',
    'tradition': 'conservation',
    'security': 'conservation',
    'power': 'self-enhancement',
    'achievement': 'self-enhancement',
    'hedonism': 'hhedonism',
    'stimulation': 'opennes-to-change',
    'self-direction': 'opennes-to-change'
}

schwartz_hier_neg = {
    'universalism': 'self-enhancement',
    'benevolence': 'self-enhancement',
    'conformity': 'opennes-to-change',
    'tradition': 'opennes-to-change',
    'security': 'opennes-to-change',
    'power': 'self-transcendence',
    'achievement': 'self-transcendence',
    'hedonism': '',
    'stimulation': 'conservation',
    'self-direction': 'conservation'
}

In [225]:
def rowMaxThemes(row, th):
    tScores = row[3:].astype(np.float64)
    sMax = tScores.max()
    maxThemes = tScores[(tScores<=sMax) & (tScores>=sMax-th)]
    
    #return list(maxThemes.keys())
    return list(zip(list(maxThemes.keys()), maxThemes.values))

In [226]:
def rowMinThemes(row, th):
    tScores = row[3:].astype(np.float64)
    sMin = tScores.min()
    minThemes = tScores[(tScores>=sMin) & (tScores<=sMin+th)]
    
    #return list(minThemes.keys())
    return list(zip(list(minThemes.keys()), minThemes.values))

In [227]:
def themeThreshold(row, th):
    theme = str(row[2])
    if row[theme] >= th:
        return True
    else:
        return False

In [270]:
df2 = df.copy()
max_th = 5
df2['max_themes_th'+str(max_th)] = df.apply(rowMaxThemes, axis=1, args=(max_th,))
min_th = 5
df2['min_themes_th'+str(min_th)] = df.apply(rowMinThemes, axis=1, args=(min_th,))
theme_th = 80
df2['theme_threshold_th'+str(theme_th)] = df.apply(themeThreshold, axis=1, args=(theme_th,))

In [271]:
df2

,document.id,title,theme,universalism,benevolence,conformity,tradition,security,power,achievement,hedonism,stimulation,self-direction,max_themes_th5,min_themes_th5,theme_threshold_th80
0,1,Critical thinking,universalism,49.522670,47.604883,50.989633,39.579652,28.752928,30.702778,61.130890,7.447772,0.248711,51.706620,"[(achievement, 61.130889817466915)]","[(stimulation, 0.24871079848182515)]",False
1,221,Social work,universalism,73.956331,24.425526,12.386535,3.048106,66.451233,27.560275,59.865460,10.737655,20.006899,19.260099,"[(universalism, 73.95633053412107)]","[(tradition, 3.0481061974002466)]",False
2,222,Labor rights,universalism,76.822639,62.819241,6.305191,0.000120,42.287528,45.281959,46.115006,0.196758,86.278109,26.479325,"[(stimulation, 86.27810920333991)]","[(tradition, 0.00011991054647983188), (hedonis...",False
3,223,Left-wing politics,universalism,66.274238,13.560333,0.512608,22.805877,16.102093,81.301090,39.825376,6.828770,29.786765,50.355647,"[(power, 81.30108985872236)]","[(conformity, 0.5126084542927725)]",False
4,224,Climate justice,universalism,96.270829,0.121169,0.066082,1.235794,48.480765,0.592800,57.491868,0.793439,28.115263,20.107254,"[(universalism, 96.27082914850236)]","[(benevolence, 0.12116926539076914), (conformi...",True
5,225,Labour law,universalism,68.391937,12.125260,22.560340,12.962435,55.590560,43.972425,71.510394,4.198938,23.552645,23.236121,"[(universalism, 68.39193651011864), (achieveme...","[(hedonism, 4.198938307948906)]",False
6,226,Right to housing,universalism,80.427772,7.550699,73.616564,0.003292,82.750115,31.816324,51.850497,0.000159,87.574089,0.146260,"[(security, 82.75011516567835), (stimulation, ...","[(tradition, 0.0032923163340555475), (hedonism...",True
7,227,Social law,universalism,75.720724,26.056795,17.685650,0.229041,54.938594,5.781897,74.721615,9.807221,8.630734,47.604617,"[(universalism, 75.72072438930273), (achieveme...","[(tradition, 0.22904120901640293)]",False
8,228,Solidarity,universalism,32.287511,66.735757,62.146231,30.092037,74.499337,66.413071,60.312410,17.557381,61.779806,5.848783,"[(security, 74.49933674122067)]","[(self-direction, 5.848782511882321)]",False
9,229,Essentially contested concept,universalism,35.662750,85.184936,41.548259,50.947351,30.489266,40.418570,42.291413,39.418567,1.529226,15.101290,"[(benevolence, 85.1849357723987)]","[(stimulation, 1.5292260146643708)]",False


In [272]:
# Count theme_threshold_th groupby theme
theme_threshold_count = df2.groupby('theme').sum()['theme_threshold_th'+str(theme_th)]
theme_threshold_count

theme
achievement       28.0
benevolence       11.0
conformity        18.0
hedonism          44.0
power              3.0
security          11.0
self-direction    11.0
stimulation        3.0
tradition         10.0
universalism      41.0
Name: theme_threshold_th80, dtype: float64

In [273]:
groupMaxTheme = dict(((s, dict(((s, 0) for s in schwartz))) for s in schwartz))
groupMinTheme = dict(((s, dict(((s, 0) for s in schwartz))) for s in schwartz))

groupMaxHier = dict(((s, dict(((s, 0) for s in list(schwartz_hier.keys())))) for s in schwartz))
groupMinHier = dict(((s, dict(((s, 0) for s in list(schwartz_hier.keys())))) for s in schwartz))

hierGroupMaxHier = dict(((s, dict(((s, 0) for s in list(schwartz_hier.keys())))) for s in list(schwartz_hier.keys())))
hierGroupMinHier = dict(((s, dict(((s, 0) for s in list(schwartz_hier.keys())))) for s in list(schwartz_hier.keys())))

for idx, row in df2.iterrows():
    theme = row['theme']
    for mx_theme in row['max_themes_th'+str(max_th)]:
        groupMaxTheme[theme][mx_theme[0]] += 1
        groupMaxHier[theme][schwartz_hier_pos[mx_theme[0]]] += 1
        hierGroupMaxHier[schwartz_hier_pos[theme]][schwartz_hier_pos[mx_theme[0]]] += 1
    for mn_theme in row['min_themes_th'+str(min_th)]:
        groupMinTheme[theme][mn_theme[0]] += 1
        groupMinHier[theme][schwartz_hier_pos[mn_theme[0]]] += 1
        hierGroupMinHier[schwartz_hier_pos[theme]][schwartz_hier_pos[mn_theme[0]]] += 1

In [285]:
pd.DataFrame(groupMaxHier)

,achievement,benevolence,conformity,hedonism,power,security,self-direction,stimulation,tradition,universalism
conservation,9,25,27,7,5,14,3,1,16,29
hhedonism,1,2,1,62,0,0,2,0,0,2
opennes-to-change,7,9,1,8,3,8,21,4,3,19
self-enhancement,44,6,8,5,14,9,10,2,0,36
self-transcendence,3,23,1,7,6,7,2,2,6,84


In [283]:
groupMaxTheme

{'achievement': {'achievement': 37,
  'benevolence': 2,
  'conformity': 2,
  'hedonism': 1,
  'power': 7,
  'security': 7,
  'self-direction': 6,
  'stimulation': 1,
  'tradition': 0,
  'universalism': 1},
 'benevolence': {'achievement': 5,
  'benevolence': 22,
  'conformity': 10,
  'hedonism': 2,
  'power': 1,
  'security': 1,
  'self-direction': 2,
  'stimulation': 7,
  'tradition': 14,
  'universalism': 1},
 'conformity': {'achievement': 4,
  'benevolence': 1,
  'conformity': 22,
  'hedonism': 1,
  'power': 4,
  'security': 3,
  'self-direction': 1,
  'stimulation': 0,
  'tradition': 2,
  'universalism': 0},
 'hedonism': {'achievement': 3,
  'benevolence': 6,
  'conformity': 3,
  'hedonism': 62,
  'power': 2,
  'security': 2,
  'self-direction': 2,
  'stimulation': 6,
  'tradition': 2,
  'universalism': 1},
 'power': {'achievement': 9,
  'benevolence': 1,
  'conformity': 3,
  'hedonism': 0,
  'power': 5,
  'security': 2,
  'self-direction': 1,
  'stimulation': 2,
  'tradition': 0,
 